In [ ]:
import os
import datetime

import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
import plotly.express as px
import plotly

from api_utils import FinancialModelingPrepAPI

tqdm.pandas()

#### Setup API

In [40]:
load_dotenv()
api_key = os.getenv("API_KEY")
api = FinancialModelingPrepAPI(api_key)

#### Get S&P 500 companies

In [77]:
snp_url = f"https://financialmodelingprep.com/api/v3/sp500_constituent?apikey={api_key}"
response = api.session.get(snp_url).json()
snp_data = pd.DataFrame.from_records(response)
snp_data.head()

,symbol,name,sector,subSector,headQuarter,dateFirstAdded,cik,founded
0,APO,Apollo Global Management,Financial Services,Asset Management - Global,"New York City, New York",2024-12-23,0001858681,1990
1,LII,Lennox International,Industrials,Construction,"Richardson, Texas",2024-12-23,0001069202,1895
2,WDAY,"Workday, Inc.",Technology,Software - Application,"Pleasanton, California",2024-12-23,0001327811,2005
3,TPL,Texas Pacific Land Corporation,Energy,Oil & Gas Exploration & Production,"Dallas, Texas",2024-11-26,0001811074,1888
4,DELL,Dell Technologies,Technology,Computer Hardware,"Round Rock, Texas",2024-09-23,0001571996,2016


### Benjamin Graham's criteria for stock selection (Defensive Investor)

1. Adequate size of the enterprise
2. Strong financial condition
3. Earnings stability
4. Dividend record
5. Earnings growth
6. Moderate price/earnings ratio
7. Moderate price to assets ratio


#### 1. Adequate size of the enterprise
Adequate size of the enterprise (market cap > $2 billion in 2003 -> ~ $3,4 billion in 2024)
Reference: Inflation calculator - https://www.nerdwallet.com/calculator/inflation-calculator


In [42]:
# Initial call returns an empty list sometimes, so we need to retry until we get the data
snp_data["market_cap"] = snp_data.progress_apply(
    lambda x: api.get_market_cap(x["symbol"]), axis=1
)
print(f"Companies without market cap: {snp_data['market_cap'].isna().sum()}")

# set time limit for the while loop
time_limit = datetime.datetime.now() + datetime.timedelta(seconds=30)

# While loop to ensure all companies have a market cap
while snp_data["market_cap"].isna().any() and datetime.datetime.now() < time_limit:
    # Apply the get_market_cap function to companies without a market cap
    snp_data["market_cap"] = snp_data.progress_apply(
        lambda x: api.get_market_cap(x["symbol"])
        if pd.isna(x["market_cap"])
        else x["market_cap"],
        axis=1,
    )

# Sanity check
print(f"Companies without market cap: {snp_data['market_cap'].isna().sum()}")

100%|██████████| 503/503 [01:32<00:00,  5.41it/s]


Companies without market cap: 25


100%|██████████| 503/503 [00:04<00:00, 111.57it/s] 

Companies without market cap: 0


In [43]:
snp_data["market_cap"].describe()

count    5.030000e+02
mean     1.094061e+11
std      3.189740e+11
min      4.736278e+09
25%      1.957982e+10
50%      3.622064e+10
75%      8.136981e+10
max      3.590733e+12
Name: market_cap, dtype: float64

In [44]:
fig = px.histogram(
    snp_data,
    x="market_cap",
    title="Market capitalization of S&P 500 companies",
    nbins=3000,
)
fig.show()

In [45]:
# plotly.io.write_json(fig, 'data/market_cap.json')

In [46]:
market_cap_limit = 3421630434.78
first_criteria = snp_data[snp_data["market_cap"] >= market_cap_limit]["symbol"].unique()
print(len(first_criteria))

503


In [47]:
# plot for top 10 largest companies with their market cap
fig = px.bar(
    snp_data.sort_values(by="market_cap", ascending=False).head(10),
    x="market_cap",
    y="name",
    title="Top 10 largest companies in the S&P 500",
    orientation="h",
)
fig.show()

We can see that all companies have a market cap above 3.4 billion dollars since these are in fact the S&P 500 companies - the biggest companies in the US. This means that investing in any of these companies would satisfy the first criteria. However, if we want to use the same criteria for other indexes (say with smaller capitalization), this criteria would be useful to determine which companies are of adequate size. Note that, the limit of 3.4 billion dollars applies for 2024, so we can adjust this limit for other years.

In [48]:
plotly.io.write_json(fig, "data/10_largest_companies.json")

In [8]:
# Uncomment to save data
# snp_data.to_csv('snp_data.csv', index=False)

#### 2. Strong financial condition
Strong financial condition (current ratio > 2) - Current assets at least twice their current liabilities.

In [49]:
ratios = [api.get_ratios(ticker) for ticker in tqdm(snp_data["symbol"])]
ratios_df = pd.DataFrame.from_records(ratios)
companies_without_ratios = ratios_df[ratios_df["currentRatio"].isna()]["symbol"]
print(f"Companies without ratios: {len(companies_without_ratios)}")
# set time limit for the while loop
time_limit = datetime.datetime.now() + datetime.timedelta(seconds=15)
while len(companies_without_ratios) > 0 and datetime.datetime.now() < time_limit:
    new_ratios = [api.get_ratios(ticker) for ticker in tqdm(companies_without_ratios)]
    new_ratios_df = pd.DataFrame.from_records(new_ratios)
    ratios_df = (
        pd.concat([ratios_df, new_ratios_df])
        .drop_duplicates(subset="symbol", keep="last")
        .reset_index(drop=True)
    )
    companies_without_ratios = ratios_df[ratios_df["currentRatio"].isna()]["symbol"]

# Sanity check
print(len(ratios_df))

100%|██████████| 503/503 [01:44<00:00,  4.81it/s]


Companies without ratios: 0
503


In [50]:
ratios_df["name"] = ratios_df.apply(
    lambda x: snp_data[snp_data["symbol"] == x["symbol"]]["name"].values[0], axis=1
)

In [51]:
second_criteria = ratios_df[ratios_df["currentRatio"] > 2]["symbol"].unique()
print(len(second_criteria))

115


In [52]:
# outliers
ratios_df[ratios_df["currentRatio"] > 50]

,symbol,date,calendarYear,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,...,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,name
279,AMP,2024-12-31,2024,FY,8169.000000,8169.000000,718.571429,0.000000,0.0,0.000000,...,15.811652,18.588120,18.588120,18.588120,0.495108,2.999857,0.056252,10.632954,0.393069,Ameriprise Financial
317,MET,2024-12-31,2024,FY,84.626289,84.626289,12.930412,158.215000,0.0,158.215000,...,13.068240,12.215424,12.215424,12.215424,0.049902,0.814809,0.094519,0.000000,2.107489,MetLife
340,COF,2024-12-31,2024,FY,132.992492,132.992492,4.546547,23.629065,0.0,23.629065,...,14.366961,3.758085,3.758085,3.758085,1.399319,1.744811,0.016998,9500.350345,1.122714,Capital One
351,HBAN,2024-12-31,2024,FY,145.517588,145.517588,8.467337,278.873097,0.0,278.873097,...,12.172484,13.010810,13.010810,13.010810,0.902196,2.380266,0.006056,0.000000,1.196283,Huntington Bancshares
363,L,2024-12-31,2024,FY,147.513043,147.513043,4.704348,251.600000,0.0,251.600000,...,13.192234,7.795161,6.166552,6.166552,6.925923,1.277659,0.002948,0.000000,0.976129,Loews Corporation


In [53]:
df_current_ratio_gt_2 = ratios_df[
    (ratios_df["currentRatio"] > 2) & (ratios_df["currentRatio"] < 50)
].sort_values(by="currentRatio", ascending=False)
fig = px.bar(
    df_current_ratio_gt_2,
    x="symbol",
    y="currentRatio",
    title="Current ratio of S&P 500 companies",
    hover_name="name",
)
fig.show()

In [54]:
plotly.io.write_json(fig, "data/current_ratio.json")

In [55]:
# Uncomment to save data
# ratios_df.to_csv('ratios.csv', index=False)

#### 3. Earnings stability
- Positive earnings for each of the last 10 years. (Earnings per share). 
- Note that it is possible that some companies went public less than 10 years ago, so they are automatically excluded from this criteria. 
- Also, depending on when you run this code, you may end up with 11 years - where the average EPS per year for the 11th year will be calculated with less than four quarters. This is because we request for 44 quarters (11 years) of data - the response includes data for future quarters as well.
- Advice: If you like the business, go deeper into the data - the indicators may seem misleading. For example, Amazon is not on the list just because it has a negative EPS for one quarter in 2014.

In [112]:
# TODO: Add long term debt to net current assets (working capital) explanation + graph

In [56]:
earnings = [api.get_earnings(ticker) for ticker in tqdm(snp_data["symbol"])]
earnings = [item for sublist in earnings for item in sublist]
earnings_df = pd.DataFrame.from_records(earnings)
earnings_df = earnings_df.dropna()
earnings_df

100%|██████████| 503/503 [01:17<00:00,  6.48it/s]


,date,symbol,eps,epsEstimated,time,revenue,revenueEstimated,updatedFromDate,fiscalDateEnding
3,2025-02-04,APO,2.22,1.92,bmo,9.550000e+08,9.524547e+08,2025-02-27,2024-12-30
4,2024-11-05,APO,1.81,1.73,bmo,7.773000e+09,9.043327e+08,2025-02-20,2024-09-30
5,2024-08-01,APO,1.64,1.76,bmo,6.018000e+09,8.731757e+08,2025-02-20,2024-06-30
6,2024-05-02,APO,1.72,1.78,bmo,7.040000e+09,8.300840e+08,2025-02-20,2024-03-31
7,2024-02-08,APO,1.91,1.73,bmo,1.104600e+10,8.296172e+08,2025-02-20,2023-12-31
...,...,...,...,...,...,...,...,...,...
17725,2016-02-02,XOM,0.67,0.63,bmo,5.232100e+10,3.578494e+10,2025-02-27,2015-12-31
17726,2015-10-30,XOM,1.01,0.89,bmo,5.986600e+10,5.787555e+10,2025-02-27,2015-09-30
17727,2015-07-31,XOM,1.00,1.11,bmo,6.539500e+10,7.920960e+10,2025-02-27,2015-06-30
17728,2015-04-30,XOM,1.17,0.83,bmo,5.922800e+10,4.593944e+10,2025-02-27,2015-03-31


In [57]:
# calculate yearly eps per company
earnings_df["year"] = pd.to_datetime(earnings_df["date"]).dt.year
earnings_df = earnings_df.groupby(["symbol", "year"]).agg({"eps": "mean"}).reset_index()

In [58]:
# group by company and get all symbols which always have positive eps
earnings_df["name"] = earnings_df.apply(
    lambda x: snp_data[snp_data["symbol"] == x["symbol"]]["name"].values[0], axis=1
)
third_criteria = (
    earnings_df.groupby("symbol").filter(lambda x: all(x["eps"] > 0))["symbol"].unique()
)
print(len(third_criteria))

354


In [59]:
# plot average eps per company - for example, tech companies
# px.line(earnings_df[earnings_df['symbol'].isin(['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META'])], x='year', y='eps', color='symbol', title='Average earnings per share per year')
fig = px.line(
    earnings_df[earnings_df["symbol"].isin(third_criteria)],
    x="year",
    y="eps",
    color="name",
    title="Average earnings per share per year",
)
fig.show()

In [60]:
plotly.io.write_json(fig, "data/positive_eps.json")

#### 4. Dividend record
Uninterrupted dividends for at least the past 20 years.

In [61]:
dividends_df = pd.DataFrame()
for ticker in tqdm(snp_data["symbol"]):
    dividends = api.get_dividends(ticker)
    dividends_df_company = pd.DataFrame.from_records(dividends)
    if dividends_df_company.empty:
        dividends_df = pd.concat(
            [dividends_df, pd.DataFrame({"symbol": ticker, "num_years": 0}, index=[0])]
        )
    else:
        dividends_df_company["year"] = pd.to_datetime(
            dividends_df_company["date"]
        ).dt.year
        last_20_years = [
            year
            for year in dividends_df_company["year"].unique()
            if year in range(2005, 2025)
        ]
        num_years = len(last_20_years)
        dividends_df = pd.concat(
            [
                dividends_df,
                pd.DataFrame({"symbol": ticker, "num_years": num_years}, index=[0]),
            ]
        )

100%|██████████| 503/503 [01:25<00:00,  5.85it/s]


In [62]:
dividends_df = dividends_df.merge(snp_data[["symbol", "name"]], on="symbol")
dividends_df[dividends_df["num_years"] >= 20]

,symbol,num_years,name
1,LII,20,Lennox International
3,TPL,20,Texas Pacific Land Corporation
5,ERIE,20,Erie Indemnity
19,HUBB,20,Hubbell Incorporated
28,BG,20,Bunge Global
...,...,...,...
497,SLB,20,Schlumberger
498,SO,20,Southern Company
499,SPGI,20,S&P Global
500,UNP,20,Union Pacific Corporation


In [64]:
fourth_criteria = dividends_df[dividends_df["num_years"] >= 20]["symbol"].unique()
print(len(fourth_criteria))

251


#### 5. Earnings growth
A minimum increase of at least one-third in per-share earnings in the past 10 years using 3-year averages at the beginning and end.

In [65]:
# remove companies which don't have eps for the last 10 years
earnings_df_yearly = earnings_df.groupby("symbol").filter(lambda x: len(x) >= 10)
earnings_df_yearly

,symbol,year,eps,name
0,A,2014,0.8800,Agilent Technologies
1,A,2015,0.4325,Agilent Technologies
2,A,2016,0.4950,Agilent Technologies
3,A,2017,0.5925,Agilent Technologies
4,A,2018,0.6975,Agilent Technologies
...,...,...,...,...
4328,ZTS,2021,1.1525,Zoetis
4329,ZTS,2022,1.1825,Zoetis
4330,ZTS,2023,1.3075,Zoetis
4331,ZTS,2024,1.4400,Zoetis


In [66]:
# average first three years and last three years
earnings_growth_df = pd.DataFrame(columns=["symbol", "start_eps", "end_eps", "growth"])
for symbol in earnings_df_yearly["symbol"].unique():
    # generate dataframe (symbol, start_eps, end_eps, growth)
    earnings_growth_df = pd.concat(
        [
            earnings_growth_df,
            pd.DataFrame(
                {
                    "symbol": symbol,
                    "start_eps": earnings_df_yearly[
                        earnings_df_yearly["symbol"] == symbol
                    ]
                    .head(3)["eps"]
                    .mean(),
                    "end_eps": earnings_df_yearly[
                        earnings_df_yearly["symbol"] == symbol
                    ]
                    .tail(3)["eps"]
                    .mean(),
                },
                index=[0],
            ),
        ]
    )
earnings_growth_df["growth"] = earnings_growth_df.apply(
    lambda x: (x["end_eps"] - x["start_eps"]) / abs(x["start_eps"]), axis=1
)
earnings_growth_df["gt_33_percent"] = earnings_growth_df.apply(
    lambda x: x["growth"] > 0.33, axis=1
)
earnings_growth_df[earnings_growth_df["gt_33_percent"]].sort_values(
    "growth", ascending=False
)

/var/folders/nf/dwbr_1dd3wj7vbhhprmj9sfr0000gn/T/ipykernel_1361/1618462240.py:5: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,symbol,start_eps,end_eps,growth,gt_33_percent
0,EOG,-0.001389,3.162500,2278.000000,True
0,VRTX,0.026667,2.600833,96.531250,True
0,NVDA,0.010000,0.587500,57.750000,True
0,CHTR,-0.151944,8.577500,57.451554,True
0,OXY,0.025000,0.930000,36.200000,True
...,...,...,...,...,...
0,NI,0.321667,0.443333,0.378238,True
0,WYNN,0.988333,1.346667,0.362563,True
0,FTV,0.729167,0.976667,0.339429,True
0,HUM,2.325000,3.110000,0.337634,True


In [67]:
fifth_criteria = earnings_growth_df[earnings_growth_df["gt_33_percent"]][
    "symbol"
].unique()
print(len(fifth_criteria))

300


In [68]:
earnings_growth_fifth_criterion = earnings_growth_df[
    (earnings_growth_df["symbol"].isin(fifth_criteria))
    & (earnings_growth_df["growth"] <= 100)
].sort_values("growth", ascending=False)
earnings_growth_fifth_criterion["name"] = earnings_growth_fifth_criterion.apply(
    lambda x: snp_data[snp_data["symbol"] == x["symbol"]]["name"].values[0], axis=1
)
earnings_growth_fifth_criterion["growth"] = (
    earnings_growth_fifth_criterion["growth"] * 100
)

In [69]:
fig = px.bar(
    earnings_growth_fifth_criterion,
    x="symbol",
    y="growth",
    title="Earnings growth of S&P 500 companies (in %)",
    hover_name="name",
)
fig.show()

In [70]:
plotly.io.write_json(fig, "data/eps_growth.json")

#### 6. Moderate price/earnings ratio
- Current price is not more than 15 times average earnings of the past 3 years.
- Since the API offers only the trailing twelve months (TTM) P/E ratio, we will use that instead.

In [79]:
ratios_ttm = [api.get_ratios_ttm(ticker) for ticker in tqdm(snp_data["symbol"])]
ratios_ttm_df = pd.DataFrame.from_records(ratios_ttm)
ratios_ttm_df

100%|██████████| 503/503 [01:27<00:00,  5.72it/s]


,dividendYielTTM,dividendYielPercentageTTM,peRatioTTM,pegRatioTTM,payoutRatioTTM,currentRatioTTM,quickRatioTTM,cashRatioTTM,daysOfSalesOutstandingTTM,daysOfInventoryOutstandingTTM,...,priceEarningsRatioTTM,priceToFreeCashFlowsRatioTTM,priceToOperatingCashFlowsRatioTTM,priceCashFlowRatioTTM,priceEarningsToGrowthRatioTTM,priceSalesRatioTTM,enterpriseValueMultipleTTM,priceFairValueTTM,dividendPerShareTTM,symbol
0,0.012510,1.251014,14.805704,0.393953,0.199932,0.057983,0.057983,0.064572,0.000000,0.000000,...,14.805704,-90.227361,18.733813,18.733813,0.393953,2.646503,8.443310,4.846272,1.85000,APO
1,0.007504,0.750367,26.752723,3.801041,0.198662,1.434706,0.898043,0.316074,45.176549,72.071497,...,26.752723,27.585299,22.826237,22.826237,3.801041,4.039178,20.557287,25.390228,4.55000,LII
2,0.000000,0.000000,132.686019,-1.956880,0.000000,1.900685,1.900685,0.278118,84.350557,0.000000,...,132.686019,31.859325,28.348028,28.348028,-1.956880,8.276326,163.501524,7.725575,0.00000,WDAY
3,0.011032,1.103192,69.337340,58.816152,0.765065,8.333924,8.333924,6.123096,65.504454,0.000000,...,69.337340,481.268303,64.149531,64.149531,58.816152,44.592576,55.121538,27.794571,15.10667,TPL
4,0.016290,1.628992,18.901334,5.541075,0.299291,0.768384,0.634000,0.105556,62.896205,33.162783,...,18.901334,27.141364,14.145760,14.145760,5.541075,0.816084,17.410459,-33.856963,1.78000,DELL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,0.032471,3.247083,22.088098,-3.282748,0.671211,0.668668,0.458013,0.066904,58.579741,78.139734,...,22.088098,176.552951,9.931520,9.931520,-3.282748,3.640199,12.362118,2.927298,2.88000,SO
499,0.007052,0.705194,42.289947,4.848393,0.294316,0.854036,0.854036,0.260638,73.657704,0.000000,...,42.289947,28.941496,28.641794,28.641794,4.848393,11.336625,25.036742,4.913995,3.69000,SPGI
500,0.021544,2.154443,21.946690,11.391377,0.476212,0.765322,0.765322,0.193376,0.000000,0.000000,...,21.946690,25.354826,15.843603,15.843603,11.391377,6.162530,14.464216,8.766981,5.28000,UNP
501,0.023193,2.319260,21.234705,35.568131,0.610845,0.933595,0.824684,0.263742,52.347116,22.369837,...,21.234705,-22.468408,8.067558,8.067558,35.568131,2.950150,12.332585,2.063998,1.64250,XEL


In [82]:
ratios_ttm_df[ratios_ttm_df["peRatioTTM"] <= 15][["symbol", "peRatioTTM"]]

,symbol,peRatioTTM
0,APO,14.805704
16,BLDR,14.909733
17,JBL,13.512242
28,BG,8.871689
31,STLD,13.253066
...,...,...
485,MO,8.308711
486,MRK,13.410875
490,OXY,14.835114
497,SLB,12.996281


In [83]:
ratios_ttm_df["name"] = ratios_ttm_df.apply(
    lambda x: snp_data[snp_data["symbol"] == x["symbol"]]["name"].values[0], axis=1
)
ratios_ttm_df["sector"] = ratios_ttm_df.apply(
    lambda x: snp_data[snp_data["symbol"] == x["symbol"]]["sector"].values[0], axis=1
)

In [84]:
sixth_criteria = ratios_ttm_df[
    (ratios_ttm_df["peRatioTTM"] <= 15) & (ratios_ttm_df["peRatioTTM"] > 0)
]["symbol"].unique()
print(len(sixth_criteria))

110


In [85]:
ratios_ttm_df_sixth_criteria = ratios_ttm_df[
    ratios_ttm_df["symbol"].isin(sixth_criteria)
][["symbol", "name", "sector", "peRatioTTM"]].sort_values("peRatioTTM", ascending=False)
ratios_ttm_df_sixth_criteria

,symbol,name,sector,peRatioTTM
299,ELV,Elevance Health,Healthcare,14.993879
16,BLDR,Builders FirstSource,Industrials,14.909733
175,MLM,Martin Marietta Materials,Basic Materials,14.857004
262,AIZ,Assurant,Financial Services,14.852174
490,OXY,Occidental Petroleum,Energy,14.835114
...,...,...,...,...
159,SYF,Synchrony Financial,Financial Services,6.702765
470,F,Ford Motor Company,Consumer Cyclical,6.294605
33,ACGL,Arch Capital Group,Financial Services,6.019819
353,APA,APA Corporation,Energy,3.422874


In [86]:
fig = px.bar(
    ratios_ttm_df_sixth_criteria,
    x="symbol",
    y="peRatioTTM",
    color="sector",
    hover_name="name",
)
fig.show()

In [87]:
plotly.io.write_json(fig, "data/pe_ratio.json")

#### 7. Moderate price to assets ratio
- Price should not be more than 1.5 times the book value last reported.
- This ratio is calculated by dividing the company's current stock price per share by its book value per share (BVPS).

In [113]:
# TODO: P/E * P/B < 22.5 (same usage as P/B <= 1.5)

In [88]:
ratios_df[ratios_df["priceToBookRatio"] <= 1.5][["symbol", "priceToBookRatio"]]

,symbol,priceToBookRatio
2,WDAY,0.000000
4,DELL,-25.362252
7,SW,0.000000
26,AXON,0.000000
27,FICO,-49.817540
...,...,...
465,ED,1.405837
469,EXC,1.402359
470,F,0.878381
485,MO,-40.140402


In [89]:
seventh_criteria = ratios_df[ratios_df["priceToBookRatio"] <= 1.5]["symbol"].unique()
seventh_criteria_df = ratios_df[ratios_df["symbol"].isin(seventh_criteria)]
seventh_criteria_df["sector"] = seventh_criteria_df.apply(
    lambda x: snp_data[snp_data["symbol"] == x["symbol"]]["sector"].values[0], axis=1
)
seventh_criteria_df = seventh_criteria_df[seventh_criteria_df["priceToBookRatio"] > 0]
print(len(seventh_criteria))

117


/var/folders/nf/dwbr_1dd3wj7vbhhprmj9sfr0000gn/T/ipykernel_1361/1488887280.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [90]:
fig = px.bar(
    seventh_criteria_df,
    x="symbol",
    y="priceToBookRatio",
    color="sector",
    hover_name="name",
)
fig.show()

In [91]:
plotly.io.write_json(fig, "data/pb_ratio.json")

### Combine all criteria

In [92]:
# Store all dataframes
date = datetime.datetime.now().strftime("%Y-%m-%d")
snp_data.to_csv(f"data/snp_data_{date}.csv", index=False)
ratios_df.to_csv(f"data/ratios_{date}.csv", index=False)
earnings_df.to_csv(f"data/earnings_{date}.csv", index=False)
earnings_growth_df.to_csv(f"data/earnings_growth_{date}.csv", index=False)
dividends_df.to_csv(f"data/dividends_{date}.csv", index=False)
ratios_ttm_df.to_csv(f"data/ratios_ttm_{date}.csv", index=False)

In [93]:
# compile a dataframe of shape (symbol, criteria1, criteria2, ..., criteria7), where value of criteria is True if the company meets the criteria
criteria_df = (
    snp_data[["symbol", "name"]]
    .merge(
        pd.DataFrame({"symbol": first_criteria, "large_market_cap": True}),
        on="symbol",
        how="left",
    )
    .fillna(False)
)
criteria_df = criteria_df.merge(
    pd.DataFrame({"symbol": second_criteria, "double_current_ratio": True}),
    on="symbol",
    how="left",
).fillna(False)
criteria_df = criteria_df.merge(
    pd.DataFrame({"symbol": third_criteria, "earnings_stability": True}),
    on="symbol",
    how="left",
).fillna(False)
criteria_df = criteria_df.merge(
    pd.DataFrame({"symbol": fourth_criteria, "dividend_record": True}),
    on="symbol",
    how="left",
).fillna(False)
criteria_df = criteria_df.merge(
    pd.DataFrame({"symbol": fifth_criteria, "earnings_growth": True}),
    on="symbol",
    how="left",
).fillna(False)
criteria_df = criteria_df.merge(
    pd.DataFrame({"symbol": sixth_criteria, "moderate_pe_ratio": True}),
    on="symbol",
    how="left",
).fillna(False)
criteria_df = criteria_df.merge(
    pd.DataFrame({"symbol": seventh_criteria, "moderate_pb_ratio": True}),
    on="symbol",
    how="left",
).fillna(False)
# count satisfied criteria
criteria_df["num_criteria"] = criteria_df[
    [
        "large_market_cap",
        "double_current_ratio",
        "earnings_stability",
        "dividend_record",
        "earnings_growth",
        "moderate_pe_ratio",
        "moderate_pb_ratio",
    ]
].sum(axis=1)
criteria_df.sort_values("num_criteria", ascending=False)

/var/folders/nf/dwbr_1dd3wj7vbhhprmj9sfr0000gn/T/ipykernel_1361/3656524461.py:5: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/nf/dwbr_1dd3wj7vbhhprmj9sfr0000gn/T/ipykernel_1361/3656524461.py:7: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/nf/dwbr_1dd3wj7vbhhprmj9sfr0000gn/T/ipykernel_1361/3656524461.py:9: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option

,symbol,name,large_market_cap,double_current_ratio,earnings_stability,dividend_record,earnings_growth,moderate_pe_ratio,moderate_pb_ratio,num_criteria
28,BG,Bunge Global,True,True,True,True,True,True,True,7
289,MTB,M&T Bank,True,True,True,True,True,True,True,7
294,STT,State Street Corporation,True,True,True,True,True,True,True,7
31,STLD,Steel Dynamics,True,False,True,True,True,True,True,6
317,MET,MetLife,True,True,True,True,True,True,False,6
...,...,...,...,...,...,...,...,...,...,...
172,RCL,Royal Caribbean Group,True,False,False,False,False,False,False,1
80,LVS,Las Vegas Sands,True,False,False,False,False,False,False,1
56,CZR,Caesars Entertainment,True,False,False,False,False,False,False,1
91,DOW,Dow Inc.,True,False,False,False,False,False,False,1


In [94]:
criteria_df.to_csv(f"data/all_criteria_{date}.csv", index=False)

In [115]:
import plotly.graph_objects as go


# Convert column names to have line breaks for better fitting
def format_header(col_name):
    # Replace underscores with line breaks
    return col_name.replace("_", "<br>")


# Format all column headers
formatted_headers = [format_header(col) for col in criteria_df.columns]

# Adjust column widths - first column narrower, second wider for names, others standard
columnwidth = [60, 150] + [100] * (len(criteria_df.columns) - 2)

fig = go.Figure(
    data=go.Table(
        columnwidth=columnwidth,
        header=dict(
            values=list(formatted_headers),
            fill_color="rgb(153, 128, 250)",  # Purple similar to #9980FA
            font=dict(color="white", size=12, family="Arial, sans-serif"),
            align="center",
            height=60,
            line=dict(color="white", width=1.5),
        ),
        cells=dict(
            values=criteria_df.sort_values(by="num_criteria", ascending=False)
            .transpose()
            .values.tolist(),
            fill_color="rgb(246, 229, 141)",  # Yellow similar to #f6e58d
            font=dict(color="rgb(50, 50, 50)", size=12, family="Arial, sans-serif"),
            align=["center", "left"]
            + ["center"] * (len(criteria_df.columns) - 2),  # Align name column left
            height=30,
            line=dict(color="white", width=1),
        ),
    )
)

# Update layout for better visual appearance
fig.update_layout(
    paper_bgcolor="white",
    plot_bgcolor="white",
    margin=dict(l=10, r=10, t=10, b=10),
    height=450,
    width=1000,
)

fig.show()

In [117]:
plotly.io.write_json(fig, "data/all_criteria.json")